<a href="https://colab.research.google.com/github/dviva1972/denvlaiva/blob/master/DLL_HW_11_CV_NOISE_hw.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## DLL

## Домашняя работа 11 | CV-3

## Иванов Денис

###Задание 
Возьмите MNIST, обучите на нем LeNet

Обучите шум, который при добавлении к оригинальной картинке MNIST, заставит модель ошибаться в пользу заданного класса

### 1. Импорт библиотек

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.autograd import Variable

from PIL import Image
import numpy as np
import random

In [ ]:
dev = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(f"work on {(torch.cuda.get_device_name() if dev.type == 'cuda' else 'cpu')}")

work on cpu


### 2. Загрузка данных

In [ ]:
def load_data(train_batch_size, test_batch_size):
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=True, download=True,
                       transform=transforms.Compose([
                           transforms.Resize((32, 32)),
                           transforms.ToTensor(),
                           transforms.Normalize((0.1307,), (0.3081,))
                       ])),
        batch_size=train_batch_size, shuffle=True)

    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('data', train=False, transform=transforms.Compose([
            transforms.Resize((32, 32)),
            transforms.ToTensor(),
            transforms.Normalize((0.1307,), (0.3081,))
        ])),
        batch_size=test_batch_size, shuffle=True)

    return (train_loader, test_loader)

### 3. MNIST to LeNet

https://www.kaggle.com/vincentman0403/pytorch-v0-3-1b-on-mnist-by-lenet

#### 3.1. Конфигурация и базовые функции 

##### https://www.kaggle.com/vincentman0403/pytorch-v0-3-1b-on-mnist-by-lenet

In [ ]:
class LeNet(nn.Module):  # try LeNet from DLL11
    def __init__(self):
        super(LeNet, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)      
        self.fc1 = nn.Linear(16 * 5 * 5, 120)          
        self.fc2 = nn.Linear(120, 84)         
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = F.max_pool2d(F.relu(self.conv1(x)), 2)
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)   
        x = x.view(x.size()[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

In [ ]:
def train(model, optimizer, epoch, train_loader, log_interval):
    model.train()
    loss_fn = torch.nn.CrossEntropyLoss()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data), Variable(target)
        optimizer.zero_grad()
        output = model(data)
        loss = loss_fn(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train set, Epoch {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                       100. * batch_idx / len(train_loader),
                loss.data#[0]
                ))

In [ ]:
def test(model, epoch, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    loss_fn = torch.nn.CrossEntropyLoss(size_average=False)
    for data, target in test_loader:
        data, target = Variable(data), Variable(target)
        output = model(data)
        test_loss = loss_fn(output, target)
        pred = np.argmax(output.data, axis=1)
        correct = correct + np.equal(pred, target.data).sum()
    test_loss /= len(test_loader.dataset)
    print('\nTest set, Epoch {} , Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

### 3.2.  Обучение LeNet

In [ ]:
torch.manual_seed(123)

model = LeNet()

lr = 0.0005
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum)

train_batch_size = 128
test_batch_size = 1000
train_loader, test_loader = load_data(train_batch_size, test_batch_size)

epochs = 10
log_interval = 100
for epoch in range(1, epochs + 1):
    train(model, optimizer, epoch, train_loader, log_interval=log_interval)
    test(model, epoch, test_loader)

Train set, Epoch 1 [0/60000 (0%)]	Loss: 2.297626
Train set, Epoch 1 [12800/60000 (21%)]	Loss: 2.295653
Train set, Epoch 1 [25600/60000 (43%)]	Loss: 2.295305
Train set, Epoch 1 [38400/60000 (64%)]	Loss: 2.295582
Train set, Epoch 1 [51200/60000 (85%)]	Loss: 2.292281


/usr/local/lib/python3.7/dist-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))



Test set, Epoch 1 , Average loss: 0.2289, Accuracy: 1252/10000 (13%)

Train set, Epoch 2 [0/60000 (0%)]	Loss: 2.280790
Train set, Epoch 2 [12800/60000 (21%)]	Loss: 2.283422
Train set, Epoch 2 [25600/60000 (43%)]	Loss: 2.279119
Train set, Epoch 2 [38400/60000 (64%)]	Loss: 2.277004
Train set, Epoch 2 [51200/60000 (85%)]	Loss: 2.273926

Test set, Epoch 2 , Average loss: 0.2265, Accuracy: 2586/10000 (26%)

Train set, Epoch 3 [0/60000 (0%)]	Loss: 2.261367
Train set, Epoch 3 [12800/60000 (21%)]	Loss: 2.252381
Train set, Epoch 3 [25600/60000 (43%)]	Loss: 2.239599
Train set, Epoch 3 [38400/60000 (64%)]	Loss: 2.229549
Train set, Epoch 3 [51200/60000 (85%)]	Loss: 2.223428

Test set, Epoch 3 , Average loss: 0.2217, Accuracy: 3368/10000 (34%)

Train set, Epoch 4 [0/60000 (0%)]	Loss: 2.218352
Train set, Epoch 4 [12800/60000 (21%)]	Loss: 2.216971
Train set, Epoch 4 [25600/60000 (43%)]	Loss: 2.185018
Train set, Epoch 4 [38400/60000 (64%)]	Loss: 2.171707
Train set, Epoch 4 [51200/60000 (85%)]	Loss: 2

In [ ]:
model.eval()

LeNet(
  (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

### 4.  Обучение шума

#### 4.1. генерация случайного шума

In [ ]:
def noise_func(size, n):
    h_points   = [random.choice(range(size)) for _ in range(n)]
    w_points   = [random.choice(range(size)) for _ in range(n)]
    matr_noise = np.ones((size, size))
    for i in range(n):
        matr_noise[w_points[i]][h_points[i]] = 0
    return torch.from_numpy(matr_noise).float()

#### 4.2. обучение случайного шума на  достижение схожести (любой)  произвольной картинки MNIST c целевым классом - 8 

In [ ]:
class Net1(nn.Module):
    def __init__(self):
        super().__init__()
        self.line = nn.Linear(1024,1024)
    def forward(self, input):
        x = self.line(input.flatten())
        x = x.view((32,32))
        return x

model_noise  = Net1()

In [ ]:
train_loader, test_loader = load_data(10000, 10000)

optim   = torch.optim.SGD(model_noise.parameters(), lr = 0.00002)
crit    =  nn.CrossEntropyLoss()
n_iters = 10
target  = 8

train_scores = []

model_noise.train()

for iter in range(n_iters):
    for batch_idx, (data, target) in enumerate(train_loader):
        X, y = Variable(data), Variable(target)
        optim.zero_grad()
        for noise_iter in range(25):
            noise_matrix = noise_func(32, 100)
            pred = model(X*model_noise(noise_matrix)).float()
            y = y * 0 + target
            loss = crit(pred, y) 
            train_score = torch.sum(torch.argmax(pred, axis=1) == y
                                    ).item() / y.shape[0]
            train_scores.append(train_score)
            if train_score > best_score:
                best_score = train_score
                best_loss  = loss
            loss.backward() 
            optim.step()    
                       
    print(f'iter {iter}, loss = {loss}, train score = {np.mean(train_scores)}')

iter 0, loss = 0.8646298050880432, train score = 0.6577173333333334
iter 1, loss = 0.6982666254043579, train score = 0.734982
iter 2, loss = 0.598858118057251, train score = 0.7682935555555555
iter 3, loss = 0.5670689940452576, train score = 0.787813
iter 4, loss = 0.5209929943084717, train score = 0.8007902666666668
iter 5, loss = 0.5264170169830322, train score = 0.810158
iter 6, loss = 0.5017315149307251, train score = 0.8173024761904761
iter 7, loss = 0.4829837381839752, train score = 0.8230334166666667
iter 8, loss = 0.4802733361721039, train score = 0.827715111111111
iter 9, loss = 0.44495707750320435, train score = 0.8316031333333334
